# Worksheet 08

Name: Toby Ueno
UID: U23639106

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [24]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.625369379780688, 5.593499764776039, 3.656993839764148, 6.169565777469046, 3.6257096340860007, 3.6184872133395607, 4.430368527670962, 3.6081139526849286, 3.802138172773215, 5.286872913255044]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [25]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[8.881896886883743, 9.108461654011254, 6.99193023750115, 9.047805376761723, 8.853389035537289, 8.083528309671033, 9.020415302602558, 7.5284488233713, 9.39915206906757, 9.488320403246501]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [26]:
data = []
for i in range(10):
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[9.488320403246501, 9.39915206906757, 5.286872913255044, 3.802138172773215, 7.5284488233713, 3.6081139526849286, 4.430368527670962, 3.6184872133395607, 3.6257096340860007, 9.020415302602558]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

$\mu_1=5, \mu_2=8, \sigma_1=\sigma_2=1, p=0.5$

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [27]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[0])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.286872913255044, 3.802138172773215, 3.6081139526849286, 4.430368527670962, 3.6184872133395607, 3.6257096340860007]
[9.488320403246501, 9.39915206906757, 7.5284488233713, 9.020415302602558]
P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 4.061948402301618,  mean_2 = 8.859084149571983
var_1 = 0.3827670997246763,  var_2 = 23.6335688109587


Means are fairly close, variances are not

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [28]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( (pdf_i[0] * prob_s[0]) / prob_x )
    prob_s1_x.append( (pdf_i[1] * prob_s[1]) / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  9.488320403246501
probability of observing that point if it came from cluster 0 =  2.3769292280640694e-44
probability of observing that point if it came from cluster 1 =  0.016874341272947554
point =  9.39915206906757
probability of observing that point if it came from cluster 0 =  6.288056640283848e-43
probability of observing that point if it came from cluster 1 =  0.016875916331263525
point =  5.286872913255044
probability of observing that point if it came from cluster 0 =  0.006224927692826825
probability of observing that point if it came from cluster 1 =  0.01668859401754535
point =  3.802138172773215
probability of observing that point if it came from cluster 0 =  0.8278086374581849
probability of observing that point if it came from cluster 1 =  0.016498284595000856
point =  7.5284488233713
probability of observing that point if it came from cluster 0 =  1.613702267610116e-18
probability of observing that point if it came from cluster 1 =  0.016853589087795585
point =

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [29]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [sum([x[0] * ((x[1] - mean[0]) ** 2) for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * ((x[1] - mean[0]) ** 2) for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 3.9174756646869193,  mean_2 = 8.27810378054405
var_1 = 0.22955428300530673,  var_2 = 21.42449096436387


Means are closer to true values, variances and probabilities are not

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [30]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( (pdf_i[0] * prob_s[0]) / prob_x )
    prob_s1_x.append( (pdf_i[1] * prob_s[1]) / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  9.488320403246501
probability of observing that point if it came from cluster 0 =  2.2555523824574032e-128
probability of observing that point if it came from cluster 1 =  0.018591167826144825
point =  9.39915206906757
probability of observing that point if it came from cluster 0 =  2.5969502416150565e-124
probability of observing that point if it came from cluster 1 =  0.018595378068571925
point =  5.286872913255044
probability of observing that point if it came from cluster 0 =  3.254348875680212e-08
probability of observing that point if it came from cluster 1 =  0.018440245600310677
point =  3.802138172773215
probability of observing that point if it came from cluster 0 =  1.5318156318776226
probability of observing that point if it came from cluster 1 =  0.018218883990661617
point =  7.5284488233713
probability of observing that point if it came from cluster 0 =  3.222142400002283e-54
probability of observing that point if it came from cluster 1 =  0.01860945659964938
poi

The probabilities of each point are more polarized

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [33]:
assigns = [(x[0], 0 if x[1] > x[2] else 1) for x in zip(data, prob_s0_x, prob_s1_x)]
print(assigns)

[(9.488320403246501, 1), (9.39915206906757, 1), (5.286872913255044, 1), (3.802138172773215, 0), (7.5284488233713, 1), (3.6081139526849286, 0), (4.430368527670962, 0), (3.6184872133395607, 0), (3.6257096340860007, 0), (9.020415302602558, 1)]
